In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8)) # | (DataAcc['cod_distrito'] == 5) 
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

/tmp/ipykernel_4542/466131193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc['Accidente'] = True


In [4]:
# Observar hora exacta de accidente
data1=DataAcc.loc['2020-08-03':'2020-08-03']
data1.head(60)

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-08-03 18:00:00,2020S010858,8.0,FUENCARRAL-EL PARDO,Colisión frontal,True


In [5]:
DataAcc = DataAcc.reset_index()

In [6]:
def ModTime(ds):
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute
    if (hora != 23) & (minuto >= 25):   # Revisar si esta bien esta condición
        date = date.replace(hour=hora+1,minute=0)
    else:
        date = date.replace(minute=0)
    return (date)
DataAcc['ds'] = DataAcc['ds'].apply(ModTime)
DataAcc = DataAcc.drop_duplicates(subset = ['ds']) # Contabilizar en una nueva columna las repeticiones del index

In [7]:
DataAcc = DataAcc.set_index('ds')
Prueba = DataAcc.loc['2020-08-03':'2020-08-03']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-08-03 18:00:00,2020S010858,8.0,FUENCARRAL-EL PARDO,Colisión frontal,True


In [8]:
DataAcc = DataAcc.reset_index()

In [9]:
def ds1(ds):
    global fila  
    global status
    global status2
    date = pd.to_datetime(ds)    
    mes = date.month
    dia = date.day
    hora = date.hour
    try:
        if (hora != 23):
            date2 = date.replace(hour=hora+1,minute=0)
        else:
            date2 = date.replace(day=dia+1,hour=0)  
    except:
            date2 = date.replace(month=mes+1, day=1, hour=0)     
    
    consulta1 = DataAcc.loc[fila+1]    
    celdaDs1 = consulta1['ds']
    celdaAcc1 = consulta1['Accidente']
    
    
    if fila < (len(DataAcc)-2):
        fila += 1
    
    if (date2 == celdaDs1) and (status == False):        
        status = True
        try:
            if (hora < 22):
                date = date.replace(hour=hora+2,minute=0)
            elif (hora == 22):
                date = date.replace(day=dia+1,hour=0)
            elif (hora == 23):
                date = date.replace(day=dia+1,hour=1)
        except:            
            if (hora == 22):
                date = date.replace(month=mes+1, day=1, hour=0) 
            elif (hora == 23):
                date = date.replace(month=mes+1, day=1, hour=1)             

    elif (date2 == celdaDs1) and (status == True):
        status2 = True
        try:
            if (hora < 21):
                date = date.replace(hour=hora+3,minute=0)
            elif (hora == 21):
                date = date.replace(day=dia+1,hour=0)
            elif (hora == 22):
                date = date.replace(day=dia+1,hour=1)
            elif (hora == 23):
                date = date.replace(day=dia+1,hour=2)
        except:            
            if (hora == 21):
                date = date.replace(month=mes+1, day=1, hour=0) 
            elif (hora == 22):
                date = date.replace(month=mes+1, day=1, hour=1) 
            elif (hora == 23):
                date = date.replace(month=mes+1, day=1, hour=2) 
                
    elif (status2 == True):
        date=date
        status2=False
        
        
    else: 
        status=False        
        try:
            if (hora != 23):   # Revisar si esta bien esta condición
                date = date.replace(hour=hora+1,minute=0)
            else:
                date = date.replace(day=dia+1,hour=0)  
        except:
            date = date.replace(month=mes+1, day=1, hour=0)  
    
    return (date)

status=False
status2=False
fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ds1)

In [10]:
def ds2(ds):
    date = pd.to_datetime(ds)
    mes = date.month
    dia = date.day
    hora = date.hour    
    try:
        if (hora < 22):   # Revisar si esta bien esta condición
            date = date.replace(hour=hora+2,minute=0)
        elif (hora == 22):
            date = date.replace(day=dia+1,hour=0)
        elif (hora == 23):
            date = date.replace(day=dia+1,hour=1)
    except:
        if (hora == 22):
            date = date.replace(month=mes+1, day=1, hour=0) 
        elif (hora == 23):
            date = date.replace(month=mes+1, day=1, hour=1) 
    return (date)
DataAcc['ds2'] = DataAcc['ds'].apply(ds2)

def Accidente1(tipo_accidente):
    global valor
    if (tipo_accidente == 'Colisión múltiple'): 
        valor = 1
#     elif (tipo_accidente == 'Vuelco'):
#         valor = 2
    elif (tipo_accidente == 'Choque contra obstáculo fijo'):
        valor = 3
    elif (tipo_accidente == 'Colisión frontal') | (tipo_accidente == 'Colisión lateral') | (tipo_accidente == 'Colisión fronto-lateral'):
        valor = 10
    elif (tipo_accidente == 'Alcance'):
        valor = 4
    elif (tipo_accidente == 'Atropello a persona'):
        valor = 5
    elif (tipo_accidente == 'Atropello a animal'):
        valor = 2
#     elif (tipo_accidente == 'Solo salida de la vía'):
#         valor = 8
    else:
        valor = 0
    
    return (valor)


In [11]:
# DataAcc.tail(60)
# DataAcc = DataAcc.set_index('ds')
# data1=DataAcc.loc['2020-08-01':'2020-08-05']
# data1

# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Choque contra obstáculo fijo']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Colisión múltiple']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Colisión frontal']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Colisión lateral']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Colisión fronto-lateral']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Alcance']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Atropello a persona']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Atropello a animal']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Vuelco']
# prueba = DataAcc.loc[DataAcc.loc[:,'tipo_accidente'] == 'Solo salida de la vía']
# prueba

In [12]:
fila=0
DataAcc['Accidente1'] = DataAcc['tipo_accidente'].apply(Accidente1)
DataAcc['Accidente2'] = 1

In [13]:
# prueba = DataAcc.loc[DataAcc.loc[:,'Accidente1'] == 0]
# prueba

In [14]:
DataAcc = DataAcc.set_index('ds')
Acc_ds = DataAcc.loc[:,['Accidente']]
DataAcc = DataAcc.set_index('ds1')
Acc_ds1 = DataAcc.loc[:,['Accidente1']]
DataAcc = DataAcc.set_index('ds2')
Acc_ds2 = DataAcc.loc[:,['Accidente2']]

Acc_ds1 = Acc_ds1.reset_index() 
Acc_ds1 = Acc_ds1.drop_duplicates(subset = ['ds1'])
Acc_ds1 = Acc_ds1.set_index('ds1') 

In [15]:
Acc_ds1
Data=Acc_ds1.loc['2020-08-24':'2020-08-24']
Data

/tmp/ipykernel_4542/2909898902.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  Data=Acc_ds1.loc['2020-08-24':'2020-08-24']


,Accidente1
ds1,
2020-08-24 03:00:00,3
2020-08-24 05:00:00,3


In [16]:
# Acc_ds.to_csv("Acc_ds.csv")
# Acc_ds1.to_csv("Acc_ds1.csv")
# Acc_ds2.to_csv("Acc_ds2.csv")